# Algorithm với ngôn ngữ Python

## Bài 1: Đếm tần suất cơn ho và thở khò khè

### Bs. Lê Ngọc Khả Nhi

Thân chào các bạn, algorithms là đề tài thú vị khi làm việc với một ngôn ngữ lập trình. Việc học algorithm mang lại lợi ích quan trọng, ngay cả cho người bác sĩ, vì nó rèn luyện cho người học khả năng tận dụng một cách tối ưu những công cụ có sẵn để giải quyết một vấn đề thực tiễn; mà trên hành trình nghiên cứu y học ta sẽ gặp rất nhiều vấn đề, và nếu không may - vấn đề đó chưa từng được người đi trước giải quyết, bạn sẽ phải tự mình làm việc này.

# Bài toán cần giải quyết

Trong thí dụ sau, một nhóm nghiên cứu chế tạo ra một mô hình Deep learning phân tích tín hiệu âm thanh từ microphone cho phép phát hiện những cơn ho và cơn khó thở khò khè của bệnh nhân. Giả định kết quả của mô hình là một chuỗi giá trị gồm 3 con số: 0 = Bình thường, 1 = Ho, 2 = Thở khò khè. Các bác sĩ muốn chuyển kết quả của mô hình thành 1 điểm số lâm sàng : số cơn ho trung bình / 1 giờ và số cơ thở khỏ khè / 10 phút; được xác định một cách đơn giản bằng cách lấy tổng số cơn ho hoặc khó thở và chia cho tổng thời gian ghi âm tính bằng giờ. Họ cũng quy ước rằng khi có nhiều labels liên tiếp = 1 thì được tính như 1 cơn ho kéo dài, tương tự cho thở khò khè. 

Như vậy ta có một bài toán như sau:

Dữ liệu đầu vào: chuỗi dữ liệu số nguyên (integer list hoặc numpy array)

Mục tiêu: Đếm tần suất các cụm giá trị lặp lại của 0,1,2, độ dài ngắn nhất = 1, dài nhất = toàn bộ chuỗi

Sau đây, Nhi sẽ đề xuất giải pháp cho bài toán.

Đầu tiên, ta mô phỏng chuỗi giá trị đầu vào bằng numpy.random, với độ dài 600 giây

In [56]:
import numpy as np

np.random.seed(123)
seq = np.random.choice([0,1,2],
                        size = 3600, 
                        p = [0.5,0.2,0.3])

seq[0:10]

array([1, 0, 0, 1, 2, 0, 2, 1, 0, 0])

# Câu hỏi thứ 1

Câu hỏi quan trọng nhất trong bài toán này, đó là: làm cách nào ta có thể gom những giá trị lặp lại liên tục thành một biến cố kéo dài, và phân lập biến cố này với trạng thái trước và sau nó ?

Giải pháp cho câu hỏi này đơn giản không ngờ: Ta chỉ cần phát hiện tất cả vị trí chuyển tiếp từ trạng thái này sang trạng thái khác, hoặc câu hỏi: giá trị ở vị trí t+1 có khác với vị trí t hay không ? Để giải quyết, ta chỉ cần so sánh 2 array seq[1:] và seq[:-1]

Ta thử làm nháp:

In [4]:
x = np.array([1,0,0,2,1,2])

mask = x[1:] != x[:-1]

mask

array([ True, False,  True,  True,  True])

Như vậy, object mask chính là tất cả vị trí chuyển đổi, thí dụ từ 1 thành 0, từ 0 thành 2, từ 2 thành 1 và cuối cùng từ 1 thành 2.

Nếu ta chồng mask này lên chuỗi x[:-1], ta sẽ phân lập được tất cả những cụm biến cố đã xảy ra.

In [5]:
x[:-1][mask]

array([1, 0, 2, 1])

Tuy nhiên, ta không quên biến cố cuối cùng (vị trí -1), ta chỉ cần ghép nó vào list kết quả

In [7]:
np.append(x[:-1][mask], x[-1])

array([1, 0, 2, 1, 2])

Ta đưa giải pháp này vào hàm isolate_events()

In [24]:
def isolate_events(x):
    
    # nếu x rỗng hay chỉ có 1 phần tử, kết quả là chính nó
    
    if len(x) <= 1:
        return x
    
    # Nếu x chỉ chứa toàn 1 giá trị, trả kết quả là giá trị đó
    
    elif len(np.unique(x)) == 1:
        return x[0]
    
    # Nếu không, phân lập các biến cố
    else:
        mask = x[1:] != x[:-1]
        return np.append(x[:-1][mask], x[-1])

Thử hàm này lần lượt cho các trường hợp:

In [25]:
# Không có điểm chuyển tiếp nào

x = np.array([1,1,1,1,1,1])

isolate_events(x)

1

In [26]:
# x rỗng

x = np.array([])

isolate_events(x)

array([], dtype=float64)

In [27]:
# x chi có 1 phần tử

x = np.array([2])

isolate_events(x)

array([2])

In [28]:
# Điểm chuyển tiếp nằm ngay trước giá trị cuối cùng

x = np.array([0,1,1,1,1,2])

isolate_events(x)

array([0, 1, 2])

In [30]:
# Điểm chuyển tiếp nằm rải rác

x = np.array([0,1,1,0,0,2,1])

isolate_events(x)

array([0, 1, 0, 2, 1])

# Câu hỏi thứ 2:

Như vậy ta đã giải quyết xong phần khó nhất của bài toán, bước tiếp theo là đếm tần suất xuất hiện của các giá trị trong danh sách các biến cố. Đây là 1 bài toán binary search kinh điển, ta có rất nhiều cách để giải nó, nhưng Nhi sẽ dùng giải pháp hàm đệ quy:

Hàm này sẽ đối chiếu 1 target (thí dụ: 0,1,2) với nội dung trong list seq, và xuất ra tổng số lần tìm thấy target.

In [32]:
def recursive_count(seq, target):
    
    if len(seq) == 0:
        return(0)
    
    if seq[0] not in target:
        return recursive_count(seq[1:], target)
    else:
        return 1 + recursive_count(seq[1:], target)

test 2 hàm  kết hợp trên x:

In [37]:
recursive_count(isolate_events(x), [0])

2

In [38]:
recursive_count(isolate_events(x), [1])

2

In [39]:
recursive_count(isolate_events(x), [2])

1

# Đóng gói

Mọi thứ đều tốt ! Bây giờ ta chỉ cần đóng gói lại 2 hàm nói trên vào 1 hàm count_events(), cho phép đếm tần suất cơn ho (1), thở khò khè (2) và giai đoạn bình thường (0) và lưu kết quả vào 1 dictionary:

In [51]:
def count_events(seq = None, labels = [0,1,2]):
    
    # Tạo place holder là 1 dictionary có key = target label
    count_dict = {lb:[] for lb in labels}
    
    # Vòng lặp: đếm biến cố cho mỗi key
    
    for lb in count_dict:
        event_list = isolate_events(seq)
        total_event = recursive_count(event_list, [lb])
        count_dict[lb] = total_event
        
    return count_dict

Ta test hàm này trên x:

In [58]:
x = np.array([0,0,0,1,1,0,0,2,2,2,2,1,1,2,2,1,0,0,2,1,0,1])

count_events(x, labels = [0,1,2])

{0: 4, 1: 5, 2: 3}

Bây giờ ta áp dụng hàm này trên chuỗi dữ liệu seq có độ dài 3600:

Ghi chú: Mặc định thì Python chỉ cho phép 1 hàm tự gọi chính nó 1000 lượt, nếu chuỗi seq quá dài, có thể bạn cần nâng hạn mức này lên cao hơn:

In [61]:
import sys
sys.setrecursionlimit(1500)

In [59]:
count_events(seq, labels = [0,1,2])

{0: 918, 1: 566, 2: 766}

Kết quả nằm gọn trong dictionary, theo đó ta có 918 cụm trạng thái bình thường (0), 566 cụm biến cố là cơn ho (1) và 766 cụm biến cố thở khò khè.

# Tóm tắt

Những chiêu thức sử dụng trong bài bao gồm:

+ Hàm đệ quy (recursive function)

+ dictionary comprehension

+ numpy array slicing, indexing và append

+ boolean mask

+ Vòng lặp for

Chúc các bạn thực hành vui và hẹn gặp lại.